# PRUEBA FINAL
## TRATAMIENTO DE DATOS

In [11]:
import os
import cv2
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense


In [12]:
train_folder = 'CarneDataset/train'
test_folder = 'CarneDataset/test'



In [13]:
train_images = []
train_labels = []
for label in os.listdir(train_folder):
    label_folder = os.path.join(train_folder, label)
    for filename in os.listdir(label_folder):
        img_path = os.path.join(label_folder, filename)
        img = cv2.imread(img_path)
        if img is not None:
            train_images.append(img)
            train_labels.append(label)

In [14]:
train_images = np.array(train_images)
train_labels = np.array(train_labels)

In [15]:
label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(train_labels)

In [16]:
train_images, val_images, train_labels_encoded, val_labels_encoded = train_test_split(train_images, train_labels_encoded, test_size=0.2, random_state=42)

In [17]:
train_images = train_images.astype('float32') / 255.0
val_images = val_images.astype('float32') / 255.0

In [18]:
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=train_images.shape[1:]))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(len(label_encoder.classes_), activation='softmax'))

In [19]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [20]:
model.fit(train_images, train_labels_encoded, epochs=10, batch_size=32, validation_data=(val_images, val_labels_encoded))

Epoch 1/10
41/41 [==============================] - 71s 2s/step - loss: 1.8716 - accuracy: 0.5555 - val_loss: 1.1048 - val_accuracy: 0.6422
Epoch 2/10
41/41 [==============================] - 65s 2s/step - loss: 0.9229 - accuracy: 0.6970 - val_loss: 0.7678 - val_accuracy: 0.7003
Epoch 3/10
41/41 [==============================] - 66s 2s/step - loss: 0.5920 - accuracy: 0.7995 - val_loss: 0.5420 - val_accuracy: 0.8165
Epoch 4/10
41/41 [==============================] - 66s 2s/step - loss: 0.3604 - accuracy: 0.8753 - val_loss: 0.6815 - val_accuracy: 0.8165
Epoch 5/10
41/41 [==============================] - 66s 2s/step - loss: 0.1992 - accuracy: 0.9419 - val_loss: 0.4333 - val_accuracy: 0.8349
Epoch 6/10
41/41 [==============================] - 67s 2s/step - loss: 0.1407 - accuracy: 0.9610 - val_loss: 0.4189 - val_accuracy: 0.8563
Epoch 7/10
41/41 [==============================] - 65s 2s/step - loss: 0.0538 - accuracy: 0.9908 - val_loss: 0.5064 - val_accuracy: 0.8257
Epoch 8/10
41/41 [==

In [21]:
test_images = []
test_labels = []
for label in os.listdir(test_folder):
    label_folder = os.path.join(test_folder, label)
    for filename in os.listdir(label_folder):
        img_path = os.path.join(label_folder, filename)
        img = cv2.imread(img_path)
        if img is not None:
            test_images.append(img)
            test_labels.append(label)

In [22]:
test_images = np.array(test_images)
test_labels = np.array(test_labels)

In [23]:
test_images = test_images.astype('float32') / 255.0

In [24]:
test_labels_encoded = label_encoder.transform(test_labels)

In [25]:
predictions = model.predict(test_images)
predicted_labels = np.argmax(predictions, axis=1)

26/26 [==============================] - 7s 258ms/step


In [26]:
confusion = confusion_matrix(test_labels_encoded, predicted_labels)
print('Matriz de confusión del modelo:')
print(confusion)

Matriz de confusión del modelo:
[[  0   0   0   0   0   0   1   0]
 [  0  25   3  12   4   0   4   0]
 [  0   0  69   0  11   1  16   0]
 [  0   1   3  34   1   0   6   0]
 [  0   1  18   0 431   0   4   5]
 [  0   0   1   0   3  15   0   0]
 [  0   0  15   5   8   0  86   0]
 [  0   0   0   0  17   0   0  10]]


In [27]:
train_predictions = model.predict(train_images)
train_predicted_labels = np.argmax(train_predictions, axis=1)
train_confusion = confusion_matrix(train_labels_encoded, train_predicted_labels)
print('Matriz de confusión del error en entrenamiento:')
print(train_confusion)

41/41 [==============================] - 11s 269ms/step
Matriz de confusión del error en entrenamiento:
[[  0   0   0   0   0   0   1   0]
 [  0  51   0   0   0   0   0   0]
 [  0   0 165   0   0   0   0   0]
 [  0   0   0  82   0   0   0   0]
 [  0   0   0   0 761   0   0   0]
 [  0   0   0   0   0  30   0   0]
 [  0   0   0   0   0   0 170   0]
 [  0   0   0   0   0   0   0  47]]


In [28]:
test_predictions = model.predict(test_images)
test_predicted_labels = np.argmax(test_predictions, axis=1)
test_confusion = confusion_matrix(test_labels_encoded, test_predicted_labels)
print('Matriz de confusión del error en prueba:')
print(test_confusion)

26/26 [==============================] - 7s 278ms/step
Matriz de confusión del error en prueba:
[[  0   0   0   0   0   0   1   0]
 [  0  25   3  12   4   0   4   0]
 [  0   0  69   0  11   1  16   0]
 [  0   1   3  34   1   0   6   0]
 [  0   1  18   0 431   0   4   5]
 [  0   0   1   0   3  15   0   0]
 [  0   0  15   5   8   0  86   0]
 [  0   0   0   0  17   0   0  10]]
